# Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Данные взяты с сайта kaggle
[банковские транзакции](https://www.kaggle.com/datasets/shivamb/bank-customer-segmentation)

In [ ]:
import pandas as pd
from datetime import datetime  # Критически важный импорт
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lesson 14/bank_clients_transactions.csv')

In [ ]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


# Приведение столбцов к стилю camel_case

In [ ]:
import re

def camel_case_column_name(column_name):
    # Заменяем спецсимволы и разделители на пробелы
    cleaned = re.sub(r'[^a-zA-Z0-9]', ' ', column_name)
    # Разделяем на слова по пробелам и CamelCase
    words = re.split(r' +|(?<=[a-z0-9])(?=[A-Z])', cleaned)
    # Собираем CamelCase, игнорируя пустые строки
    return ''.join([word.capitalize() for word in words if word.strip()])

# Переименовываем столбцы
df.columns = [camel_case_column_name(col) for col in df.columns]

# Выводим результат
df.head()

,TransactionId,CustomerId,CustomerDob,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmountInr
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [ ]:
df.columns

Index(['TransactionId', 'CustomerId', 'CustomerDob', 'CustGender',
       'CustLocation', 'CustAccountBalance', 'TransactionDate',
       'TransactionTime', 'TransactionAmountInr'],
      dtype='object')

# Приведение типов данных

In [ ]:
# Преобразование дат с правильным форматом
# Преобразование даты рождения с коррекцией века
df['CustomerDob'] = pd.to_datetime(
    df['CustomerDob'],
    format='%d/%m/%y',
    errors='coerce'
)

# Функция коррекции дат в будущем
def correct_future_dates(dates_series):
    current_year = pd.Timestamp.now().year

    # Определяем даты, которые явно ошибочны (будущее или слишком старые)
    future_mask = (dates_series.dt.year > current_year) | (dates_series.dt.year < 1900)

    # Для ошибочных дат: вычитаем 100 лет
    corrected_dates = dates_series.where(
        ~future_mask,
        dates_series - pd.DateOffset(years=100)
    )

    # Второй проход для проверки
    final_mask = corrected_dates.dt.year > current_year
    return corrected_dates.where(~final_mask, pd.NaT)

# Применяем коррекцию
df['CustomerDob'] = correct_future_dates(df['CustomerDob'])

df['TransactionDate'] = pd.to_datetime(
    df['TransactionDate'],
    format='%d/%m/%y',
    errors='coerce'
)

In [ ]:
df.head()

,TransactionId,CustomerId,CustomerDob,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmountInr
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,181156,1762.5


# Поиск дублей и пропусков в данных

In [ ]:
f"Количество дубликатов перед удалением: {df.duplicated().sum()}"


'Количество дубликатов перед удалением: 0'

In [ ]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   TransactionId         1048567 non-null  object        
 1   CustomerId            1048567 non-null  object        
 2   CustomerDob           987831 non-null   datetime64[ns]
 3   CustGender            1047467 non-null  object        
 4   CustLocation          1048416 non-null  object        
 5   CustAccountBalance    1046198 non-null  float64       
 6   TransactionDate       1048567 non-null  datetime64[ns]
 7   TransactionTime       1048567 non-null  int64         
 8   TransactionAmountInr  1048567 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 72.0+ MB


In [ ]:
# Проверка пропусков
df.isnull().sum()

,0
TransactionId,0
CustomerId,0
CustomerDob,60736
CustGender,1100
CustLocation,151
CustAccountBalance,2369
TransactionDate,0
TransactionTime,0
TransactionAmountInr,0


In [ ]:
# 3. Создание новых признаков даты рождения
df['IsDobMissing'] = df['CustomerDob'].isna().astype(int)
current_year = datetime.now().year
df['CustomerAge'] = (current_year - df['CustomerDob'].dt.year).astype('Int16')  # Int16 для поддержки NA

# 4. Заполнение пропусков

# Для пола
gender_mode = df['CustGender'].mode()[0]
df['GenderImputed'] = df['CustGender'].isna().astype(int)
df['CustGender'] = df['CustGender'].fillna(gender_mode)

# Для локации
location_mode = df['CustLocation'].mode()[0]
df['CustLocation'] = df['CustLocation'].fillna(location_mode)

# Для баланса с группировкой
df['CustAccountBalance'] = df.groupby(['CustLocation', 'CustGender'])['CustAccountBalance'] \
                            .transform(lambda x: x.fillna(x.median()))

# 5. Оптимизация типов данных
dtype_mapping = {
    'CustGender': 'category',
    'CustLocation': 'category',
    'IsDobMissing': 'int8',
    'GenderImputed': 'int8',
    'CustomerAge': 'Int16'
}

df = df.astype(dtype_mapping)

# 6. Финализация и проверка
print("\nОбновленная структура данных:")
print(f"Всего строк: {len(df):,}")
print("\nТипы данных:")
print(df.dtypes)
print("\nОстаток пропусков:")
print(df.isna().sum())



Обновленная структура данных:
Всего строк: 1,048,567

Типы данных:
TransactionId                   object
CustomerId                      object
CustomerDob             datetime64[ns]
CustGender                    category
CustLocation                  category
CustAccountBalance             float64
TransactionDate         datetime64[ns]
TransactionTime                  int64
TransactionAmountInr           float64
IsDobMissing                      int8
CustomerAge                      Int16
GenderImputed                     int8
dtype: object

Остаток пропусков:
TransactionId               0
CustomerId                  0
CustomerDob             60736
CustGender                  0
CustLocation                0
CustAccountBalance        188
TransactionDate             0
TransactionTime             0
TransactionAmountInr        0
IsDobMissing                0
CustomerAge             60736
GenderImputed               0
dtype: int64


In [ ]:
# 3.1 Заполнение CustomerAge
median_age_by_gender = df.groupby("CustGender")["CustomerAge"].transform("median")
df["CustomerAge"] = df["CustomerAge"].fillna(median_age_by_gender)
df["CustomerAge"] = df["CustomerAge"].fillna(df["CustomerAge"].median()).astype("Int16")

# 4.1 Доработка заполнения баланса
df["CustAccountBalance"] = df.groupby(["CustLocation", "CustGender"])["CustAccountBalance"] \
    .transform(lambda x: x.fillna(x.median()))
df["CustAccountBalance"] = df["CustAccountBalance"].fillna(df["CustAccountBalance"].median())

# Проверка
print("\nОстаток пропусков после доработки:")
print(df.isna().sum())

<ipython-input-13-c9647ff6691c>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_age_by_gender = df.groupby("CustGender")["CustomerAge"].transform("median")
<ipython-input-13-c9647ff6691c>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["CustAccountBalance"] = df.groupby(["CustLocation", "CustGender"])["CustAccountBalance"] \



Остаток пропусков после доработки:
TransactionId               0
CustomerId                  0
CustomerDob             60736
CustGender                  0
CustLocation                0
CustAccountBalance          0
TransactionDate             0
TransactionTime             0
TransactionAmountInr        0
IsDobMissing                0
CustomerAge                 0
GenderImputed               0
dtype: int64


## Создание цветовой палитры для графиков

In [ ]:
DARK_BLUES_10 = [
    '#081923',  # Самый темный (ночной синий)
    '#0b2738',
    '#0e354d',
    '#114462',
    '#145277',  # Базовые оттенки
    '#1a6b99',
    '#2084bb',
    '#329dcd',
    '#4ab6df',  # Светлые акценты
    '#62cff1'   # Самый светлый (ледяной синий)
]
LIGHT_BLUES_10 = [
    '#2a4b6a',   # Темно-синий (базовый)
    '#3a6b8c',
    '#4a8cae',
    '#5aa3c1',
    '#6ab9d4',   # Средние тона
    '#7acfe7',
    '#8adefa',
    '#9ae8ff',
    '#aaf2ff',   # Светлые пастельные
    '#bafcff'    # Очень светлый (аква)
]

# Исследовательский анализ данных

In [ ]:
df

,TransactionId,CustomerId,CustomerDob,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmountInr,IsDobMissing,CustomerAge,GenderImputed
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,0,31,0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,0,68,0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,0,29,0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,0,52,0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,181156,1762.5,0,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-04-08,M,NEW DELHI,7635.19,2016-09-18,184824,799.0,0,35,0
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,183734,460.0,0,33,0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,183313,770.0,0,36,0
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,184706,1000.0,0,47,0


In [ ]:
df.columns

Index(['TransactionId', 'CustomerId', 'CustomerDob', 'CustGender',
       'CustLocation', 'CustAccountBalance', 'TransactionDate',
       'TransactionTime', 'TransactionAmountInr', 'IsDobMissing',
       'CustomerAge', 'GenderImputed'],
      dtype='object')

## Анализ количества клиентов и транзакций

In [ ]:
# сколько уникальных клиентов?
# какое количество транзакций было совершено?
# сколько в среднем транзакций приходится на одного клиента?

In [ ]:
# Анализ клиентов и транзакций
unique_clients_total = df['CustomerId'].nunique()
total_transactions = len(df)
transactions_per_client = total_transactions / unique_clients_total

print('Уникальных клиентов:', unique_clients_total)
print('Всего транзакций:', total_transactions)
print('Среднее число транзакций на клиента:', round(transactions_per_client, 2))

Уникальных клиентов: 884265
Всего транзакций: 1048567
Среднее число транзакций на клиента: 1.19


## Анализ баланса и транзакций

In [ ]:
import plotly.express as px
import numpy as np

# Создаем сегменты баланса
if 'BalanceSegment' not in df.columns:
    balance_bins = [0, 1000, 10000, 1e5, 1e6, np.inf]
    balance_labels = ['<1K', '1K-10K', '10K-100K', '100K-1M', '>1M']
    df['BalanceSegment'] = pd.cut(
        df['CustAccountBalance'],
        bins=balance_bins,
        labels=balance_labels
    )

# Создаем сегменты транзакций
if 'TransactionSegment' not in df.columns:
    transaction_bins = [0, 1000, 10000, 1e5, 1e6, np.inf]
    transaction_labels = ['<1K', '1K-10K', '10K-100K', '100K-1M', '>1M']
    df['TransactionSegment'] = pd.cut(
        df['TransactionAmountInr'],
        bins=transaction_bins,
        labels=transaction_labels
    )

# Настройки размера графиков
CHART_WIDTH = 1000  # Меняйте эти значения
CHART_HEIGHT = 800  # для регулировки размера

# 1. Сегментация баланса
fig_balance_segmented = px.pie(
    df,
    names='BalanceSegment',
    title='Сегментация баланса клиентов',
    color_discrete_sequence=px.colors.sequential.Blues_r,
    hole=0.4,
    width=CHART_WIDTH,
    height=CHART_HEIGHT
).update_layout(
    margin=dict(l=50, r=50, b=50, t=100)  # Настройка отступов
).update_traces(  # Вывод подписей внутри сегментов
    textinfo='percent+label'
)


# 2. Сегментация транзакций
fig_transaction_segmented = px.pie(
    df,
    names='TransactionSegment',
    title='Сегментация транзакций клиентов',
    color_discrete_sequence=px.colors.sequential.Blues_r,
    hole=0.4,
    width=CHART_WIDTH,  # Пример кастомизации ширины
    height=CHART_HEIGHT
).update_traces(
    textinfo='percent+label'
)

# Добавляем аннотации
for fig in [fig_balance_segmented, fig_transaction_segmented]:
    fig.update_layout(
        hoverlabel=dict(bgcolor='white'),
        plot_bgcolor='rgba(240,240,240,0.9)'
    )
    fig.add_annotation(
        text=f"Всего данных: {len(df):,}",
        xref='paper', yref='paper',
        x=0.05, y=1.04,
        showarrow=False
    )

fig_balance_segmented.show()

Output hidden; open in https://colab.research.google.com to view.

Большинство клиентов (45%) имеют баланс 10к-100к.

In [ ]:
fig_transaction_segmented.show()

Output hidden; open in https://colab.research.google.com to view.

Львиная доля транзакций (71,6%) меньше 1к.

In [ ]:
# 3. Комбинированный график с зумом
fig_combined = px.histogram(
    df.query('TransactionAmountInr < 20000'),  # Фильтр выбросов
    x='TransactionAmountInr',
    nbins=100,
    title='Детальное распределение (до 20K INR)',
    labels={'TransactionAmountInr': 'Сумма транзакции'},
    color_discrete_sequence=['#4c94c0'],
    width=CHART_WIDTH,  # Пример кастомизации ширины
    height=CHART_HEIGHT
).update_layout(xaxis_range=[0, 10000])

fig_combined.show()

Output hidden; open in https://colab.research.google.com to view.

Более детальный анализ транзакций показал, что сумма 295к транзакций меньше 200.

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Создаем субплотовую структуру
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        'Распределение баланса клиентов',
        'Распределение сумм транзакций'
    )
)

# 1. Ящик с усами для баланса
fig.add_trace(
    go.Box(
        y=df['CustAccountBalance'],
        name='Баланс',
        marker_color='#1f77b4',
        boxpoints=False,  # Скрываем выбросы для читаемости
        hoverinfo='y',
        boxmean=True  # Показываем среднее
    ),
    row=1, col=1
)

# 2. Ящик с усами для транзакций
fig.add_trace(
    go.Box(
        y=df['TransactionAmountInr'],
        name='Транзакции',
        marker_color='#4c94c0',
        boxpoints=False,
        hoverinfo='y',
        boxmean=True
    ),
    row=1, col=2
)

# Настройка логарифмической шкалы
fig.update_layout(
    title_text="Ящики с усами с логарифмической шкалой",
    yaxis_type="log",
    yaxis2_type="log",
    height=500,
    showlegend=False,
    plot_bgcolor='rgba(245,245,245,1)',
    margin=dict(l=50, r=50, b=50, t=50),
    yaxis=dict(
        title='Баланс (INR, log scale)',
        gridcolor='white'
    ),
    yaxis2=dict(
        title='Сумма транзакций (INR, log scale)',
        gridcolor='white'
    )
)

# Добавляем кнопки для переключения масштаба
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=[
                dict(
                    label="Логарифмическая шкала",
                    method="update",
                    args=[{"visible": [True, True]}, {"yaxis.type": "log", "yaxis2.type": "log"}]
                ),
                dict(
                    label="Линейная шкала",
                    method="update",
                    args=[{"visible": [True, True]}, {"yaxis.type": "linear", "yaxis2.type": "linear"}]
                )
            ],
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.5,
            xanchor="center",
            y=1.2,
            yanchor="top"
        )
    ]
)

# Обновляем аннотации с полной статистикой
stats_balance = df['CustAccountBalance'].describe()
stats_trans = df['TransactionAmountInr'].describe()

annotations = [
    # Статистика для баланса
    dict(
        x=0.04, y=0.7,
        xref='paper', yref='paper',
        text=(
            f"<b>Баланс (INR):</b><br>"
            f"Mean: {stats_balance['mean']:,.0f}<br>"
            f"Median: {stats_balance['50%']:,.0f}<br>"
            f"Q1: {stats_balance['25%']:,.0f}<br>"
            f"Q3: {stats_balance['75%']:,.0f}<br>"
            f"Min: {stats_balance['min']:,.0f}<br>"
            f"Max: {stats_balance['max']:,.0f}"
        ),
        showarrow=False,
        align='left',
        font=dict(size=9),
        bordercolor='#c7c7c7',
        borderwidth=1
    ),

    # Статистика для транзакций
    dict(
        x=0.95, y=0.7,
        xref='paper', yref='paper',
        text=(
            f"<b>Транзакции (INR):</b><br>"
            f"Mean: {stats_trans['mean']:,.0f}<br>"
            f"Median: {stats_trans['50%']:,.0f}<br>"
            f"Q1: {stats_trans['25%']:,.0f}<br>"
            f"Q3: {stats_trans['75%']:,.0f}<br>"
            f"Min: {stats_trans['min']:,.0f}<br>"
            f"Max: {stats_trans['max']:,.0f}"
        ),
        showarrow=False,
        align='right',
        font=dict(size=9),
        bordercolor='#c7c7c7',
        borderwidth=1
    )
]

# Обновляем layout с новыми аннотациями
fig.update_layout(
    annotations=annotations,
    margin=dict(t=150)  # Увеличиваем верхний отступ
)

# Добавляем подсказки при наведении
fig.update_traces(
    hovertemplate=(
        "<b>%{yaxis.title.text}</b><br>" +
        "Значение: %{y:,.0f}<extra></extra>"
    )
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

Статистический анализ баланса показывает, что большинство клиентов имеют низкий баланс (медиана - 16,8к), при этом есть небольшое количество состоятельных клиентов. Медианная сумма транзакции составляет 459.

In [ ]:
# Вычисление выбросов баланса по правилу Тьюки
Q1_balance = stats_balance['25%']
Q3_balance = stats_balance['75%']
IQR_balance = Q3_balance - Q1_balance
outliers_balance = df[df['CustAccountBalance'] > Q3_balance + 1.5*IQR_balance]

print(f"Выбросы баланса: {len(outliers_balance)} записей (> {Q3_balance + 1.5*IQR_balance:,.0f} INR)")

# Вычисление выбросов транзакций по правилу Тьюки
Q1_trans = stats_trans['25%']
Q3_trans = stats_trans['75%']
IQR_trans = Q3_trans - Q1_trans
outliers_trans = df[df['TransactionAmountInr'] > Q3_trans + 1.5*IQR_trans]

print(f"Выбросы транзакций: {len(outliers_trans)} записей (> {Q3_trans + 1.5*IQR_trans:,.0f} INR)")

Выбросы баланса: 141189 записей (> 136,587 INR)
Выбросы транзакций: 112134 записей (> 2,758 INR)


## Анализ возраста клиентов

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np


# 2. Статистические показатели
stats = df['CustomerAge'].describe(percentiles=[.25, .5, .75, .95])
age_table = go.Figure(go.Table(
    header=dict(values=['Метрика', 'Значение'],
                fill_color='#4c94c0',
                font=dict(color='white')),
    cells=dict(values=[
        ['Среднее', 'Медиана', 'Станд. отклонение', 'Минимум', 'Максимум', 'Q1', 'Q3', '95% перцентиль'],
        [f"{stats['mean']:.1f} лет",
         f"{stats['50%']:.1f} лет",
         f"{stats['std']:.1f}",
         f"{stats['min']:.0f}",
         f"{stats['max']:.0f}",
         f"{stats['25%']:.0f}",
         f"{stats['75%']:.0f}",
         f"{stats['95%']:.0f}"]
    ],
    fill_color='#f0f8ff')
))

# 1. Возрастные сегменты
age_bins = [0, 18, 25, 35, 45, 55, 65, 100]
age_labels = ['<18', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']

df['AgeGroup'] = pd.cut(
    df['CustomerAge'],
    bins=age_bins,
    labels=age_labels
)

fig_segments = px.pie(
    df,
    names='AgeGroup',
    title='Возрастные группы клиентов',
    color_discrete_sequence=px.colors.sequential.Blues_r,
    hole=0.4,
    category_orders={'AgeGroup': age_labels},
    width=CHART_WIDTH,  # Пример кастомизации ширины
    height=CHART_HEIGHT
)


fig_segments.update_traces(
    textposition='inside',
    textinfo='percent+label',
    pull=[0.1 if i == df['AgeGroup'].mode()[0] else 0 for i in age_labels]
)

# Показать все графики
fig_segments.show()

# Анализ аномалий
print("\nПроверка аномальных значений:")
print(f"Клиенты младше 18 лет: {len(df[df['CustomerAge'] < 18])}")
print(f"Клиенты старше 100 лет: {len(df[df['CustomerAge'] > 100])}")

Output hidden; open in https://colab.research.google.com to view.

Основная группа клиентов относится к возрастной группе от 25 до 44 лет.

In [ ]:

# Проверка наличия столбца
if 'CustomerAge' not in df.columns:
    raise KeyError("Столбец CustomerAge отсутствует в DataFrame")

# Проверка типа данных
if not pd.api.types.is_numeric_dtype(df['CustomerAge']):
    print("Конвертация типа данных...")
    df['CustomerAge'] = pd.to_numeric(df['CustomerAge'], errors='coerce')

# Проверка пропущенных значений
print(f"Пропущенные значения в возрасте: {df['CustomerAge'].isna().sum()}")

# Удаление отрицательных возрастов и аномальных значений
df = df[(df['CustomerAge'] > 0) & (df['CustomerAge'] < 100)]

# Создание графика с обработкой ошибок
try:
    fig = px.violin(
        df,
        y='CustomerAge',
        box=True,
        points=False,  # Изменено для больших данных
        title='Распределение возраста клиентов',
        labels={'CustomerAge': 'Возраст'},
        color_discrete_sequence=['#1f77b4'],
        hover_data=['CustGender'],
        orientation='v'
    )

    fig.update_layout(
        width=CHART_WIDTH,  # Пример кастомизации ширины
        height=CHART_HEIGHT

    )

    # Автоматический расчет статистик
    stats_age = df['CustomerAge'].describe()
    annotations = [
    dict(
        x=0.025, y=0.85,
        xref='paper', yref='paper',
        text=(
            f"<b>Возраст:</b><br>"
            f"Mean: {stats_age['mean']:,.0f}<br>"
            f"Median: {stats_age['50%']:,.0f}<br>"
            f"Q1: {stats_age['25%']:,.0f}<br>"
            f"Q3: {stats_age['75%']:,.0f}<br>"
            f"Min: {stats_age['min']:,.0f}<br>"
            f"Max: {stats_age['max']:,.0f}"
        ),
        showarrow=False,
        align='left',
        font=dict(size=9),
        bordercolor='#c7c7c7',
        borderwidth=1
    )

    ]

    fig.update_layout(annotations=annotations)
    fig.show()

except Exception as e:
    print(f"Ошибка при построении графика: {str(e)}")
    print("Дополнительная информация:")
    print(f"Тип данных возраста: {df['CustomerAge'].dtype}")
    print(f"Пример значений: {df['CustomerAge'].head().tolist()}")

Output hidden; open in https://colab.research.google.com to view.

Медианный возраст клиентов - 38 лет. Возраст 75% клиентов менее 43 лет.

## Анализ распределения клиентов по полу и возрасту

In [ ]:
import plotly.express as px


# Проверка наличия необходимых столбцов
required_columns = ['CustomerId', 'CustGender']
if not all(col in df.columns for col in required_columns):
    missing = [col for col in required_columns if col not in df.columns]
    raise ValueError(f"Отсутствуют столбцы: {missing}")

# Получаем уникальных клиентов (последняя запись для каждого клиента)
unique_customers = df.groupby('CustomerId', as_index=False).last()

# Создаем график
fig = px.pie(
    unique_customers,
    names='CustGender',
    title='Распределение клиентов по полу',
    color_discrete_sequence=px.colors.sequential.Blues_r,  # Синяя последовательная палитра
    hole=0.3,
    labels={'CustGender': 'Пол'},
    width=CHART_WIDTH,  # Пример кастомизации ширины
    height=CHART_HEIGHT
)

# Настройка отображения
fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    hovertemplate="<b>%{label}</b><br>Доля: %{percent}<br>Количество: %{value}"
)

# Добавление аннотации с общим количеством
fig.add_annotation(
    text=f"Всего клиентов: {len(unique_customers):,}",
    x=0.01, y=1.05,
    showarrow=False,
    font_size=14
)

# Настройка легенды
fig.update_layout(
    legend=dict(
        title='Пол',
        orientation='h',
        yanchor='bottom',
        y=-0.2,
        xanchor='center',
        x=0.5
    ),
    margin=dict(l=50, r=50, b=50, t=80)
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

Основная доля клиентов (73,1%) - мужского пола.

In [ ]:
# Гистограмма распределения возраста по полу
if 'CustGender' in unique_customers.columns and 'CustomerAge' in unique_customers.columns:
    fig_gender_age = px.histogram(
        unique_customers,
        x='CustomerAge',
        color='CustGender',
        barmode='overlay',
        title='Распределение возраста по полу',
        color_discrete_sequence=px.colors.sequential.Blues_r,  # Синяя и голубая
        nbins=50,
        width=1000,  # или CHART_WIDTH
        height=600   # или CHART_HEIGHT
    )

    # Настройка оформления
    fig_gender_age.update_layout(
        xaxis_title="Возраст",
        yaxis_title="Количество клиентов",
        legend_title="Пол",
        hovermode="x unified"
    )

    # Добавление аннотаций
    fig_gender_age.add_annotation(
        text=f"Всего записей: {len(unique_customers):,}",
        x=0.95, y=0.95,
        xref="paper", yref="paper",
        showarrow=False
    )

    fig_gender_age.show()
else:
    print("Столбцы 'CustGender' или 'CustomerAge' отсутствуют в данных")

Output hidden; open in https://colab.research.google.com to view.

Женщины, пользующиеся банковскими услугами, более молодые, чем мужчины.

## Анализ локаций

In [ ]:
# Группировка данных для уникальных клиентов
location_counts = unique_customers['CustLocation'].value_counts().reset_index()
location_counts.columns = ['Location', 'Count']

# Визуализация 2: Топ-15 локаций (горизонтальная гистограмма)
top_locations = location_counts.head(15)
fig_bar = px.bar(
    top_locations,
    x='Count',
    y='Location',
    orientation='h',
    title='Топ-15 локаций (уникальные клиенты)',
    labels={'Count': 'Количество клиентов', 'Location': ''},
    text_auto=True,  # Автоматические подписи данных
    color='Count',
    color_continuous_scale='Blues',
    height=600
)

# Настройка оформления
fig_bar.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    hovermode='y unified',
    xaxis=dict(tickformat=","),
    plot_bgcolor='rgba(255,255,255,255)',  # Прозрачный фон графика
    paper_bgcolor='rgba(255,255,255,255)', # Прозрачный фон области
    font=dict(color='#2a3f5f'),     # Цвет текста
    margin=dict(l=120, r=20, t=40, b=20)
)

# Настройка подписей данных
fig_bar.update_traces(
    texttemplate='%{x:,}',  # Формат чисел с разделителями
    textposition='outside',  # Расположение текста снаружи
    textfont_size=12,        # Размер шрифта
    marker_line_color='#2a3f5f',  # Цвет границ столбцов
    marker_line_width=0.5    # Толщина границ
)

fig_bar.show()

Топ-5 городов по количеству клиентов - Mumbai, New Delhi, Bangalore, Gurgaon, Delhi. Покрывают 40% клиентов.

In [ ]:
# Рассчитываем кумулятивную долю
location_counts_sorted = location_counts.sort_values('Count', ascending=False)
location_counts_sorted['CumulativePct'] = (location_counts_sorted['Count'].cumsum() /
                                         location_counts_sorted['Count'].sum() * 100)

# Находим города, покрывающие 70%
n_cities = location_counts_sorted[location_counts_sorted['CumulativePct'] <= 70].index.max() + 1
selected_cities = location_counts_sorted.head(n_cities + 1)  # +1 для округления вверх

# Визуализация
fig_bar = px.bar(
    selected_cities,
    x='Count',
    y='Location',
    orientation='h',
    title=f'Города, покрывающие 70% клиентов ({len(selected_cities)} из {len(location_counts)} локаций)',
    labels={'Count': 'Количество клиентов', 'Location': ''},
    text_auto=True,
    color='Count',
    color_continuous_scale='Blues',
    height=800
)

# Настройка оформления
fig_bar.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    hovermode='y unified',
    xaxis=dict(tickformat=","),
    plot_bgcolor='rgba(255,255,255,255)',
    paper_bgcolor='rgba(255,255,255,255)',
    font=dict(color='#2a3f5f'),
    margin=dict(l=150, r=20, t=60, b=20),
    coloraxis_colorbar=dict(
        title='Количество клиентов',
        tickformat=','

    )
)

# Добавляем аннотацию с итоговым процентом
total_pct = selected_cities['CumulativePct'].iloc[-1]
fig_bar.add_annotation(
    x=0.95, y=0.05,
    xref='paper', yref='paper',
    text=f'Общий охват: {total_pct:.1f}%',
    showarrow=False,
    bgcolor='white'
)

# Настройка подписей
fig_bar.update_traces(
    texttemplate='%{x:,} (%{customdata[0]:.1f}%)',
    textposition='outside',
    textfont_size=12,
    marker_line_color='#2a3f5f',
    marker_line_width=0.5,
    customdata=selected_cities[['CumulativePct']]
)

fig_bar.show()

41 город из 9355 городов покрывают 70% клиентов.

In [ ]:
# Группируем данные по локациям
location_stats = df.groupby('CustLocation', observed=False).agg(
    TransactionCount=('TransactionId', 'nunique'),
    TotalAmount=('TransactionAmountInr', 'sum')
).reset_index()

# Топ-10 по количеству транзакций
top10_count = location_stats.nlargest(10, 'TransactionCount')
# Топ-10 по сумме транзакций
top10_amount = location_stats.nlargest(10, 'TotalAmount')

# Визуализация
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, subplot_titles=[
    'Топ-10 локаций по количеству транзакций',
    'Топ-10 локаций по сумме транзакций'
])

# График количества транзакций
fig.add_trace(
    go.Bar(
        y=top10_count['CustLocation'],
        x=top10_count['TransactionCount'],
        orientation='h',
        name='Количество',
        marker_color='#1f77b4',
        text=top10_count['TransactionCount'].apply(lambda x: f"{x:,}"),
        textposition='auto'
    ),
    row=1, col=1
)

# График суммы транзакций
fig.add_trace(
    go.Bar(
        y=top10_amount['CustLocation'],
        x=top10_amount['TotalAmount'],
        orientation='h',
        name='Сумма',
        marker_color='#4c94c0',
        text=top10_amount['TotalAmount'].apply(lambda x: f"₹{x/1e6:,.1f}M"),
        textposition='auto'
    ),
    row=1, col=2
)

# Настройка оформления
fig.update_layout(
    height=600,
    width=1200,
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, b=20, l=100),
    xaxis1=dict(title='Количество транзакций', showgrid=False),
    xaxis2=dict(title='Сумма транзакций (INR)', showgrid=False),
    yaxis1=dict(autorange="reversed"),
    yaxis2=dict(autorange="reversed"),
    font=dict(color='#2a3f5f')
)

# Добавление аннотаций
fig.add_annotation(
    x=0.5, y=1.08,
    xref="paper", yref="paper",
    text=f"Всего транзакций: {location_stats['TransactionCount'].sum():,}",
    showarrow=False,
    font_size=12
)

fig.add_annotation(
    x=0.5, y=1.03,
    xref="paper", yref="paper",
    text=f"Общая сумма: ₹{location_stats['TotalAmount'].sum()/1e9:,.1f}B",
    showarrow=False,
    font_size=12
)

fig.show()

# Вывод табличных данных
print("Топ-10 по количеству транзакций:")
print(top10_count[['CustLocation', 'TransactionCount']].to_string(index=False))

print("\nТоп-10 по сумме транзакций:")
print(top10_amount[['CustLocation', 'TotalAmount']].to_string(index=False, formatters={
    'TotalAmount': lambda x: f"₹{x:,.0f}"
}))

Топ-10 по количеству транзакций:
CustLocation  TransactionCount
      MUMBAI            103746
   NEW DELHI             84928
   BANGALORE             81555
     GURGAON             73818
       DELHI             71019
       NOIDA             32784
     CHENNAI             30009
        PUNE             25851
   HYDERABAD             23049
       THANE             21505

Топ-10 по сумме транзакций:
CustLocation  TotalAmount
      MUMBAI ₹179,928,641
   NEW DELHI ₹160,705,853
   BANGALORE ₹118,424,843
     GURGAON ₹112,094,694
       DELHI ₹106,224,940
     KOLKATA  ₹60,600,310
     CHENNAI  ₹44,637,821
       NOIDA  ₹44,463,433
        PUNE  ₹39,590,349
   HYDERABAD  ₹36,177,394


В Noida средняя сумма транзакций ниже, чем в других городах. В Kolkata клиенты совершают более дорогие транзакции.

## Анализ динамики транзакций и времени совершения транзакции

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Группировка данных по дате
daily_stats = df.groupby('TransactionDate', as_index=False).agg(
    TransactionCount=('TransactionId', 'count'),
    TotalAmount=('TransactionAmountInr', 'sum')
)

# Создаем фигуру с двумя осями Y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# График количества транзакций (столбцы)
fig.add_trace(
    go.Bar(
        x=daily_stats['TransactionDate'],
        y=daily_stats['TransactionCount'],
        name='Количество транзакций',
        marker_color='#1f77b4',
        opacity=0.7
    ),
    secondary_y=False
)

# График суммы транзакций (линия)
fig.add_trace(
    go.Scatter(
        x=daily_stats['TransactionDate'],
        y=daily_stats['TotalAmount'],
        name='Сумма транзакций',
        line=dict(color='#ff7f0e', width=2),
        mode='lines+markers'
    ),
    secondary_y=True
)

# Настройка оформления
fig.update_layout(
    title='Динамика транзакций по дням',
    xaxis=dict(
        title='Дата',
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        title='Количество транзакций',
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis2=dict(
        title='Сумма транзакций (INR)',
        gridcolor='lightgray',
        showgrid=True
    ),
    plot_bgcolor='white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Форматирование чисел
fig.update_yaxes(
    tickformat=",",
    secondary_y=False
)

fig.update_yaxes(
    tickprefix="₹",
    tickformat=",.0f",
    secondary_y=True
)
# Расчет скользящего среднего
daily_stats['7d_avg'] = daily_stats['TotalAmount'].rolling(7).mean()

# Добавление тренда
fig.add_trace(
    go.Scatter(
        x=daily_stats['TransactionDate'],
        y=daily_stats['7d_avg'],
        name='7-дневный тренд',
        line=dict(color='#2ca02c', width=2, dash='dot')
    ),
    secondary_y=True
)
fig.show()

С начала анализируемого периода дневной оборот по транзакциям составлял 30м при совершении 20к транзакций. С 18 сентября транзакции резко прекратились.

In [ ]:
import plotly.express as px
import pandas as pd

# Конвертация в минуты и проверка данных
df['TransactionMinutes'] = df['TransactionTime'] / 60000
print(f"Максимальное время: {df['TransactionMinutes'].max():.1f} минут")
print(f"Минимальное время: {df['TransactionMinutes'].min():.2f} минут")

# Фильтрация аномальных значений (по перцентилям)
Q1 = df['TransactionMinutes'].quantile(0.05)
Q3 = df['TransactionMinutes'].quantile(0.95)
filtered_df = df[(df['TransactionMinutes'] >= Q1) & (df['TransactionMinutes'] <= Q3)]

# Визуализация распределения
fig = px.histogram(
    filtered_df,
    x='TransactionMinutes',
    nbins=50,
    title='Распределение времени транзакций',
    labels={'TransactionMinutes': 'Время транзакции (минуты)'},
    color_discrete_sequence=['#1f77b4']
)

# Добавление метрик
mean_time = filtered_df['TransactionMinutes'].mean()
median_time = filtered_df['TransactionMinutes'].median()

fig.add_vline(
    x=mean_time,
    line_dash="dot",
    annotation_text=f"Среднее: {mean_time:.1f} мин",
    annotation_position="top"
)

fig.add_vline(
    x=median_time,
    line_dash="dash",
    annotation_text=f"Медиана: {median_time:.1f} мин",
    annotation_position="bottom"
)

# Настройка оформления
fig.update_layout(
    xaxis=dict(
        range=[0, filtered_df['TransactionMinutes'].quantile(0.99)],
        title='Время выполнения транзакции (минуты)'
    ),
    yaxis_title="Количество транзакций",
    plot_bgcolor='white',
    paper_bgcolor='white',
    bargap=0.05,
    hovermode='x'
)

fig.show()

# Статистика
print(f"\nСтатистика времени транзакций (0.05-0.95 перцентиль):")
print(f"• Среднее время: {mean_time:.2f} минут")
print(f"• Медианное время: {median_time:.2f} минут")
print(f"• 95% транзакций завершаются за {filtered_df['TransactionMinutes'].quantile(0.95):.1f} минут")

# Анализ по временным отрезкам
time_bins = [0, 1, 2, 5, 10, 30, 60]
time_labels = ['<1', '1-2', '2-5', '5-10', '10-30', '30-60']

df['TimeGroup'] = pd.cut(
    df['TransactionMinutes'],
    bins=time_bins,
    labels=time_labels
)

time_distribution = df['TimeGroup'].value_counts(normalize=True).sort_index()
print("\nРаспределение по временным группам:")
print(time_distribution.to_string())

Output hidden; open in https://colab.research.google.com to view.

95% транзакций совершаются за 3,6 минуты

## Определение самой платежеспособной группы

In [ ]:
import plotly.express as px

fig = px.bar(
    payment_analysis,
    x='AgeGroup',
    y='TotalTransactionAmount',
    title='Суммарный объем транзакций по возрастным группам',
    labels={
        'TotalTransactionAmount': 'Сумма транзакций (₹)',
        'AgeGroup': 'Возрастная группа',
        'AvgBalance': 'Средний баланс'
    },
    color='AvgBalance',
    color_continuous_scale='Blues',
    text_auto='.3s'  # Автоматическое форматирование чисел
)

# Настройка оформления
fig.update_layout(
    xaxis={
        'categoryorder': 'total descending',
        'title': 'Возрастная группа',
        'gridcolor': 'lightgray'
    },
    yaxis={
        'title': 'Сумма транзакций (₹)',
        'gridcolor': 'lightgray',
        'tickformat': ',.0f',
        'rangemode': 'tozero'
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(size=12),
    hoverlabel=dict(
        bgcolor="white",
        font_size=12
    )
)

# Форматирование подписей и цвета
fig.update_traces(
    texttemplate='₹%{y:,.0f}',
    textposition='outside',
    marker_line_color='#2a3f5f',
    marker_line_width=0.5,
    hovertemplate=(
        "<b>%{x}</b><br>"
        "Сумма: ₹%{y:,.0f}<br>"
        "Средний баланс: ₹%{marker.color:,.0f}<extra></extra>"
    )
)

# Добавление аннотации
fig.add_annotation(
    x=most_solvent['AgeGroup'].values[0],
    y=most_solvent['TotalTransactionAmount'].values[0],
    text=f"Максимум: ₹{most_solvent['TotalTransactionAmount'].values[0]:,.0f}",
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,
    bgcolor='white'
)

print(f"Анализ самой платежеспособной группы ({most_solvent['AgeGroup'].values[0]}):")
print(f"• Общая сумма транзакций: ₹{most_solvent['TotalTransactionAmount'].values[0]:,.0f}")
print(f"• Средний баланс: ₹{most_solvent['AvgBalance'].values[0]:,.0f}")
print(f"• Средний чек: ₹{most_solvent['AvgTransaction'].values[0]:,.0f}")
print(f"• Количество клиентов: {most_solvent['CustomerCount'].values[0]:,}")

fig.show()

Анализ самой платежеспособной группы (35-44):
• Общая сумма транзакций: ₹884,156,309
• Средний баланс: ₹100,969
• Средний чек: ₹1,577
• Количество клиентов: 510,471


Наиболее активными клиентами являются люди 35-44 лет. Однако, наибольший баланс у более взрослой аудитории (65+).

# Выводы

На основании проведенного исследовательского анализа данных мы обнаружили закономерности:

1.   Уникальных клиентов: 884265
2.   Всего транзакций: 1048567
3.   Среднее число транзакций на клиента: 1.19
4.   Большинство клиентов (45%) имеют баланс 10к-100к.
5.   Львиная доля транзакций (71,6%) меньше 1к.
6.   Более детальный анализ транзакций показал, что сумма 295к транзакций меньше 200.
7.   Статистический анализ баланса показывает, что большинство клиентов имеют низкий баланс (медиана - 16,8к), при этом есть небольшое количество состоятельных клиентов. Медианная сумма транзакции состовляет 459.
8.   Основная группа клиентов относится к возрастной группе от 25 до 44 лет.
9.   Медианный возраст клиентов - 38 лет. Возраст 75% клиентов менее 43 лет.
10.  Основная доля клиентов (73,1%) - мужского пола.
11.  Женщины, пользующиеся банковскими услугами более молодые, чем мужчины.
12.  Топ 5 городов по количеству клиентов - Mumbai, New Delhi, Bangalore, Gurgaon, Delhi. Покрывают 40% клиентов.
13.  41 город из 9355 городов покрывают 70% клиентов.
14.  В Noida средняя сумма транзакций ниже, чем в других городах. В Kolkata клиенты совершают более дорогие транзакции.
15.  С начала анализируемого периода дневной оборот по транзакциям составлял 30м при совершении 20к транзакций. С 18 сентября транзакции резко прекратились.
16.  95% траназкций совершаются за 3,6 минуты.
17.  Наиболее активными клиентами являются люди 35-44 лет. Однако, наибольший баланс у более взрослой аудитории (65+).

